In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gaussian(x, mu=0, sigma=1.0):
    return  np.exp(-0.5 * ((x - mu) / sigma) ** 2) / (sigma * np.sqrt(2 * np.pi))

def prior(x, mu=0.9, sigma=0.1):
    """"Symmetric prior distribution, consisting of two Gaussians, one at mu and one at -mu."""
    return 0.5*gaussian(x, mu=mu, sigma=sigma) + 0.5*gaussian(x, mu=-mu, sigma=sigma)

def prob_path(x, x1, t, mu=0.9, sigma=0.1):
    return 0.5 * gaussian(x, mu=(mu + (x1 - mu) * t), sigma=sigma*(1-t)) + \
           0.5 * gaussian(x, mu=(-mu + (x1 + mu) * t), sigma=sigma*(1-t))

p_xt_p1 = lambda x,t, mu, sigma: prob_path(x, 1.0, t,mu , sigma)
p_xt_m1 = lambda x,t, mu, sigma: prob_path(x, -1.0, t,mu , sigma)
u_m1 = lambda x, t: (- 1 - x)/(1-t)
u_p1 = lambda x, t: (+ 1 - x)/(1-t)

def u_func(x, t, mu=0.9, sigma=0.1):
    """"Calculate the flow field at time t for a given x, assuming a symmetric prior distribution consisting of two Gaussians, one at mu and one at -mu."""
    p_p1 = p_xt_p1(x, t, mu, sigma)
    p_m1 = p_xt_m1(x, t, mu, sigma)
    p = p_p1 + p_m1
    return p_p1 / p * u_p1(x, t) + p_m1 / p * u_m1(x, t)

def sample_prior(rng, n_samples, mu=0.9, sigma=0.1):
    """Sample from the prior distribution."""
    x = rng.normal(mu, sigma, n_samples)
    return x*np.sign(np.random.normal(0, 1, n_samples))

def step(xt, t_start, t_end, u_func=u_func, **kwargs):
    return (xt + (t_end - t_start) * u_func(
        t=t_start + (t_end - t_start) / 2,
        x = xt + u_func(
            x=xt,
            t=t_start,
            **kwargs) * (t_end - t_start) / 2,
        **kwargs)
    )


In [ ]:
# xt = (1-t) * x0 + t * x1

# u = x1 - x0
# = x1 - (xt - t*x1)/(1-t)
# = x1*(1-t)/(1-t) - (xt - t*x1)/(1-t)
# = (x1 - xt)/(1-t)

# Plots for one case (mu = 0.9, sigma = 0.1)

In [ ]:
# Plot of probability density moving from prior to +1.0

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 1.0, 100))
p_xt_p1_temp = prob_path(x, 1.0, t)
cp = plt.contourf(t, x, p_xt_p1_temp, cmap='viridis', extend='both', levels=np.linspace(0, 10, 100))
plt.xlabel('$t$')

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.ylabel('$x_t$')
plt.colorbar(cp, label='$p(x_t,t|x_1=+1)$')

In [ ]:
# Plot of probability density moving from prior to -1.0

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 0.99, 100))
p_xt_m1_temp = prob_path(x, -1.0, t)
cp = plt.contourf(t, x, p_xt_m1_temp, cmap='viridis', extend='both', levels=np.linspace(0, 10, 100))
plt.xlabel('$t$')

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.ylabel('$x_t$')
plt.colorbar(cp, label='$p(x_t,t|x_1=+1)$')

In [ ]:
# Plot of probability density moving from prior to both +1.0 and -1.0 (the posterior)

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 1.0, 100))
cp = plt.contourf(t, x, 0.5*p_xt_p1_temp+0.5*p_xt_m1_temp, cmap='viridis', extend='both', levels=np.linspace(0, 10, 100))
plt.xlabel('$t$')

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.ylabel('$x_t$')
plt.colorbar(cp, label='$p(x_t,t|x_1=+1)$')

In [ ]:
# Plot the flow field with destination +1.0

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 1.0, 100))
x1 = 1.0
u_p1_temp = (x1 - x) / (1 - t)
cp = plt.contourf(t, x, u_p1_temp, cmap='viridis', extend='both', levels=np.linspace(-10, 10, 101))

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.xlabel('$t$')
plt.ylabel('$x_t$')
plt.colorbar(cp, label='$u(x_t,t|x_1=+1)$')

In [ ]:
# Plot the flow field with destination -1.0

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 1.0, 100))
x1 = -1.0
u_m1_temp = (x1 - x) / (1 - t)
cp = plt.contourf(t, x, u_m1_temp, cmap='viridis', extend='both', levels=np.linspace(-10, 10, 101))

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.xlabel('$t$')
plt.ylabel('$x_t$')
plt.colorbar(cp, label='$u(x_t,t|x_1=+1)$')

In [ ]:
# Plot the flow field with destination -1.0 and +1.0 (the posterior)
# By averaging the two flow fields, weighted by the probability densities

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 1.0, 100))
u = u_func(x, t, mu=0.9, sigma=0.1)
cp = plt.contourf(t, x, u, extend='both',
                  cmap='coolwarm', levels=np.linspace(-1.5, 1.5, 61))

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.xlabel('$t$')
plt.ylabel('$x_t$')
plt.colorbar(cp, label='$u(x_t,t)$')

In [ ]:
# Plot the flow field with destination -1.0 and +1.0 (the posterior)
# By averaging the two flow fields, weighted by the probability densities

plt.figure(figsize=(8,3))

# contourplot
x, t = np.meshgrid(np.linspace(-1.5, 1.5, 300), np.linspace(0, 1.0, 100))
u = u_func(x, t, mu=0.9, sigma=0.1)
cp = plt.contourf(t, x, u, extend='both',
                  cmap='coolwarm', levels=np.linspace(-1.5, 1.5, 61))

# add dots for the two final points
plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

plt.xlabel('$t$')
plt.ylabel('$x_t$')
plt.colorbar(cp, label='$u(x_t,t)$')


# Add sample paths
n_steps = 150
time_steps = np.linspace(0, 1, n_steps + 1)
rng = np.random.default_rng(42)
xt = sample_prior(rng, 100, mu=0.9, sigma=0.1)

x_pred = np.zeros((len(xt), len(time_steps)))
x_pred[:, 0] = xt
for i in range(n_steps):
    xt = step(xt, time_steps[i], time_steps[i + 1])
    x_pred[:, i+1] = xt

plt.plot(time_steps, x_pred.T, color='black', alpha=0.2);

# Same plot, various values of mu and sigma

In [ ]:
# Plot the flow field with destination -1.0 and +1.0 (the posterior)
# By averaging the two flow fields, weighted by the probability densities
# Inluding some sample paths

mu, sigma = np.meshgrid(np.linspace(0.0, 2.0, 5), [0.01, 0.05, 0.25, 1.0])
musigma = np.stack((mu, sigma), axis=-1).reshape(-1, 2)

for mu_val, sigma_val in musigma:

    plt.figure(figsize=(8,3))

    # contourplot
    x, t = np.meshgrid(np.linspace(-2.5, 2.5, 300), np.linspace(0, 1.0, 100))
    u = u_func(x, t, mu=mu_val, sigma=sigma_val)
    cp = plt.contourf(t, x, u, extend='both',
                    cmap='coolwarm', levels=np.linspace(-15, 15, 13))

    # add dots for the two final points
    plt.scatter([1.0, 1.0], [-1, 1], color='black', s=100)

    plt.xlabel('$t$')
    plt.ylabel('$x_t$')
    plt.colorbar(cp, label='$u(x_t,t)$')

    # Add sample paths
    n_steps = 150
    time_steps = np.linspace(0, 1, n_steps + 1)
    rng = np.random.default_rng(42)
    xt = sample_prior(rng, 20, mu=mu_val, sigma=sigma_val)

    x_pred = np.zeros((len(xt), len(time_steps)))
    x_pred[:, 0] = xt
    for i in range(n_steps):
        xt = step(xt, time_steps[i], time_steps[i + 1], mu=mu_val, sigma=sigma_val)
        x_pred[:, i+1] = xt

    plt.plot(time_steps, x_pred.T, color='black', alpha=0.2)

    plt.ylim(-2.5, 2.5)
    plt.xlim(0, 1)

    plt.title(f'Flow field with mu={mu_val:.2f}, sigma={sigma_val:.2f}')
    plt.show()